In [1]:
from pyspark.sql.functions import sum, min, max, col, expr,trim, when, concat_ws, collect_set, regexp_extract

In [2]:
%run ../src/revenue_attribution_spark

In [3]:
lst = [(606067,'00C078248',292.32,'2017-01-10',None,None),
(606067,'00C078249',134.43,'2011-09-19',None,None),
(625175,'00C096203',220.60,'2017-02-13',None,None),
(625175,'00C096203',48.77,'2011-10-28',None,None),
(627438,'00C098584',157.68,'2017-02-16','2010-01-10','2016-01-10'),
(627438,'00C098585',40.44,'2011-11-02','2010-01-10','2018-01-10'),
(632875,'00C103427',975.61,'2017-02-28','2019-06-18','2020-12-31'),
(632875,'00C103427',3.30,'2011-11-14',None,None),
(655686,'00C127334',132.15,'2017-04-17',None,None),
(655685,'00R127334',7993.02,'2012-01-10',None,None),
(656182,'00C127712',130.80,'2017-04-17','2008-07-07','2012-06-20'),
(656182,'00R127712',133.68,'2012-01-11',None,None)]

In [4]:
df = spark.createDataFrame(sc.parallelize(lst), ['MeetingId','RootDocumentNumber','PartSales','OpenDate','ContractStart','ContractEnd'])
df.show()

+---------+------------------+---------+----------+-------------+-----------+
MeetingId|RootDocumentNumber|PartSales| OpenDate|ContractStart|ContractEnd|
+---------+------------------+---------+----------+-------------+-----------+
 606067| 00C078248| 292.32|2017-01-10| null| null|
 606067| 00C078249| 134.43|2011-09-19| null| null|
 625175| 00C096203| 220.6|2017-02-13| null| null|
 625175| 00C096203| 48.77|2011-10-28| null| null|
 627438| 00C098584| 157.68|2017-02-16| 2010-01-10| 2016-01-10|
 627438| 00C098585| 40.44|2011-11-02| 2010-01-10| 2018-01-10|
 632875| 00C103427| 975.61|2017-02-28| 2019-06-18| 2020-12-31|
 632875| 00C103427| 3.3|2011-11-14| null| null|
 655686| 00C127334| 132.15|2017-04-17| null| null|
 655685| 00R127334| 7993.02|2012-01-10| null| null|
 656182| 00C127712| 130.8|2017-04-17| 2008-07-07| 2012-06-20|
 656182| 00R127712| 133.68|2012-01-11| null| null|
+---------+------------------+---------+----------+-------------+-----------+

In [5]:
aggregate_invoice_df = aggregate_invoice(df)

In [6]:
assert aggregate_invoice_df.filter( col('RootDocumentNumber') == '00C096203').select('PartSales').collect()[0][0] == 269.37
assert aggregate_invoice_df.filter( col('RootDocumentNumber') == '00C098584').select('OnContract').collect()[0][0] == 1

In [7]:
attribute_revenue_df = attribute_revenue(df)

In [8]:
assert attribute_revenue_df.filter( col('MeetingId') == '632875').select('PartSales').collect()[0][0] == 978.91
assert attribute_revenue_df.filter( col('MeetingId') == '627438').select('_OnContract').collect()[0][0] == 1
assert attribute_revenue_df.filter( col('MeetingId') == '656182').select('RDN_Code').collect()[0][0] == 'C'

In [9]:
attribute_revenue_df.show()

+---------+-------------------+---------+-----------+--------+
MeetingId| RootDocumentNumber|PartSales|_OnContract|RDN_Code|
+---------+-------------------+---------+-----------+--------+
 632875| 00C103427| 978.91| 0| C|
 655686| 00C127334| 132.15| 0| C|
 627438|00C098584,00C098585| 198.12| 1| C|
 625175| 00C096203| 269.37| 0| C|
 606067|00C078248,00C078249| 426.75| 0| C|
 655685| 00R127334| 7993.02| 0| |
 656182|00C127712,00R127712| 264.48| 1| C|
+---------+-------------------+---------+-----------+--------+